save_strategy="epoch" frekuensi penyimpanan model dan evaluasi selama pelatihan.

Ambil 10% dari dataset untuk training (agar lebih cepat)

In [ ]:
!pip install transformers datasets torch accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import torch


In [7]:
# Load dataset
dataset = load_dataset("AlignmentLab-AI/agentcode")

In [8]:
# Load model dan tokenizer
model_name = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token  # eos_token as pad_token


In [ ]:
# Tokenisasi dataset
def tokenize_function(examples):
    
    combined_text = [instruction + " " + response for instruction, response in zip(examples['INSTRUCTION'], examples['RESPONSE'])]
    
    tokenized_output = tokenizer(combined_text, truncation=True, padding="max_length")
   
    tokenized_output["labels"] = tokenized_output["input_ids"].copy()
    return tokenized_output

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# Ambil 10% dari dataset untuk training (agar lebih cepat)
sample_size = 0.1  
tokenized_datasets = tokenized_datasets["train"].select(range(int(len(tokenized_datasets["train"]) * sample_size)))

In [ ]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

In [ ]:

print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['INSTRUCTION', 'RESPONSE', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 19898
    })
    test: Dataset({
        features: ['INSTRUCTION', 'RESPONSE', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2211
    })
})


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False 
)

In [ ]:
training_args = TrainingArguments(
    fp16=True,                        
    output_dir="./results",
    eval_strategy="epoch",            
    save_strategy="epoch",           
    learning_rate=2e-5,
    per_device_train_batch_size=8,    
    per_device_eval_batch_size=8,    
    num_train_epochs=2,               
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,                
    save_steps=1000,                  
    save_total_limit=2,               
    gradient_accumulation_steps=2,   
    load_best_model_at_end=True,      
    metric_for_best_model="loss",     
)


In [ ]:
early_stopping = EarlyStoppingCallback(early_stopping_patience=1)

In [ ]:
trainer = Trainer(
    model=model,                         
    args=training_args,                   
    train_dataset=tokenized_datasets["train"],  
    eval_dataset=tokenized_datasets["test"],   
    tokenizer=tokenizer,                  
    data_collator=data_collator,          
    callbacks=[early_stopping]            
)

<ipython-input-17-1e7e12b15472>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.660600,1.565376


Epoch,Training Loss,Validation Loss
1,1.660600,1.565376
2,1.587800,1.516115


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=2488, training_loss=1.7044642776538321, metrics={'train_runtime': 3383.3011, 'train_samples_per_second': 11.762, 'train_steps_per_second': 0.735, 'total_flos': 1.0398565486559232e+16, 'train_loss': 1.7044642776538321, 'epoch': 2.0})

In [19]:
trainer.save_model("./finetuned_model")


In [20]:
results = trainer.evaluate()
print(results)


{'eval_loss': 1.5161153078079224, 'eval_runtime': 67.8389, 'eval_samples_per_second': 32.592, 'eval_steps_per_second': 4.083, 'epoch': 2.0}
